In [2]:
import os
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path
from itertools import combinations

In [3]:
ROOT_DIR = Path().resolve().parent
ROOT_DIR

WindowsPath('C:/Users/90004698/PT. Medco E&P Indonesia/CODE/03_non-business-demand/06_AHP_Tools')

In [4]:
def convert_keys_to_int(obj):
    if isinstance(obj, dict):
        return {int(k) if k.isdigit() else k: v for k, v in obj.items()}
    
    return obj

path_RI_file = f'{ROOT_DIR}/data_static//Ratio_Inconsistency.json'

with open(path_RI_file, 'r') as f:
    RI = json.load(f, object_hook=convert_keys_to_int)

RI

{1: 0,
 2: 0,
 3: 0.58,
 4: 0.9,
 5: 1.12,
 6: 1.24,
 7: 1.32,
 8: 1.41,
 9: 1.45,
 10: 1.49,
 11: 1.51,
 12: 1.48,
 13: 1.56,
 14: 1.57,
 15: 1.59}

In [5]:
variable = [
    'Resource Area',
    'Permeability',
    'Surface and Subsurface Risk'
]

In [6]:
comb = combinations(variable, 2)
init_score = {}

for item in comb:
    if item != ('Permeability', 'Surface and Subsurface Risk'):
        init_score[item] = 2
    else:
        init_score[item] = 1

init_score

{('Resource Area', 'Permeability'): 2,
 ('Resource Area', 'Surface and Subsurface Risk'): 2,
 ('Permeability', 'Surface and Subsurface Risk'): 1}

In [7]:
df_score = {}

for val1 in variable:
    for val2 in variable:
        if val1 != val2:
            if (val1, val2) in list(init_score.keys()):
                df_score[(val1, val2)] = init_score[(val1, val2)]
            else:
                df_score[(val1, val2)] = 1 / init_score[(val2, val1)]
        else:
            df_score[(val1, val1)] = 1


# Convert the dictionary into a Series and Convert to a DataFrame matrix form
df_score_final = pd.Series(df_score).unstack()
df_score_final.loc['Total'] = df_score_final.sum(axis=0)
df_score_final

,Permeability,Resource Area,Surface and Subsurface Risk
Permeability,1.0,0.5,1.0
Resource Area,2.0,1.0,2.0
Surface and Subsurface Risk,1.0,0.5,1.0
Total,4.0,2.0,4.0


In [9]:
df_score_norm = df_score_final.div(df_score_final.loc['Total'], axis=1)
df_score_norm['Total'] = df_score_norm.sum(axis=1) / len(variable)
df_score_norm

,Permeability,Resource Area,Surface and Subsurface Risk,Total
Permeability,0.25,0.25,0.25,0.25
Resource Area,0.50,0.50,0.50,0.50
Surface and Subsurface Risk,0.25,0.25,0.25,0.25
Total,1.00,1.00,1.00,1.00


In [18]:
df_score_calc = df_score_final.copy().iloc[0:len(variable), :].T
df_score_calc['Total'] = df_score_norm['Total']
df_score_calc

,Permeability,Resource Area,Surface and Subsurface Risk,Total
Permeability,1.0,2.0,1.0,0.25
Resource Area,0.5,1.0,0.5,0.50
Surface and Subsurface Risk,1.0,2.0,1.0,0.25


In [21]:
df_eigen = {}
eigen_value = 0

for val in variable:
    df_eigen[val] = float(np.sum(df_score_calc[val] * df_score_calc['Total'])) / float(df_score_calc['Total'].loc[val])
    eigen_value += float(np.sum(df_score_calc[val] * df_score_calc['Total']))

df_eigen['Eigenvalue Maximum'] = eigen_value
df_eigen['CI (Consistency Index)'] = (eigen_value - len(variable)) / (len(variable) - 1)
df_eigen['CR (Consistency Ratio)'] = df_eigen['CI (Consistency Index)'] / RI[len(variable)]
pd.Series(df_eigen)

Resource Area                  3.0
Permeability                   3.0
Surface and Subsurface Risk    3.0
Eigenvalue Maximum             3.0
CI (Consistency Index)         0.0
CR (Consistency Ratio)         0.0
dtype: float64